In [ ]:
from ING.PZS.bonusove_ulohy.cv3.music_tools.MusicSynthetiser import generate_multi_octave_scale, synthesize_melody, apply_rir
from ING.PZS.bonusove_ulohy.cv3.music_tools.notes import demons_souls_theme_bass, dark_souls_theme_notes, demons_souls_theme_notes, majula_theme_melody, majula_theme_bass, majula_theme_harmony
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as ipd

![Dark Souls Menu Theme](notes/DarkSoulsMainMenuTheme.png)

In [ ]:
# just the middle part of the song, played on the guitar

tones, fs = generate_multi_octave_scale(fs=16000,
                                        duration=1.0,
                                        base_note="C",
                                        base_octave=3,   # Start from C3
                                        octave_range=4)   # Generates notes from C2 to B5

melody_notes = dark_souls_theme_notes()

# You can adjust this to control the song speed
melody = synthesize_melody(tones, fs, melody_notes, base_duration=3.5, fade_ratio=0.025)

# apply reverb
rir_path = "rirs/RIRS_NOISES/real_rirs_isotropic_noises/air_type1_air_binaural_stairway_1_1_90.wav"
melody = apply_rir(melody, rir_path, fs)

# Save to WAV
sf.write("dark_souls_theme_synth.wav", melody, fs)

# Plot waveform
plt.figure(figsize=(10, 3))
plt.plot(melody[:1000000])
plt.title("Dark Souls Menu Theme (Guitar Synth)")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

# Play the sound
ipd.Audio("dark_souls_theme_synth.wav")

![Demon Souls](notes/DemonsSouls_SoulOfAMist.png)

In [ ]:
# limited to the first instrument, played on the guitar

tones, fs = generate_multi_octave_scale(fs=16000,
                                        duration=1.0,
                                        base_note="C",
                                        base_octave=3,   # Start from C3
                                        octave_range=4)   # Generates notes from C2 to B5

melody_notes = demons_souls_theme_notes()  # Use the new function

melody = synthesize_melody(
    tones, fs, melody_notes,
    base_duration=6,  # Slow and ambient feel
    fade_ratio=0.1      # Smooth note transitions
)

# apply reverb
rir_path = "rirs/RIRS_NOISES/real_rirs_isotropic_noises/air_type1_air_binaural_stairway_1_1_90.wav"
melody = apply_rir(melody, rir_path, fs)

# Save and plot
sf.write("demons_souls_theme_synth_guitar.wav", melody, fs)


plt.figure(figsize=(10, 3))
plt.plot(melody[:1000000])
plt.title("Demon's Souls Character Creation guitar (Karplus-Strong Synth)")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

# Play the sound
ipd.Audio("demons_souls_theme_synth_guitar.wav")

In [ ]:
# limited to the first instrument, played on the piano

tones, fs = generate_multi_octave_scale(fs=16000,
                                        duration=1.0,
                                        base_note="C",
                                        base_octave=3,   # Start from C3
                                        octave_range=4,  # Generates notes from C2 to B5
                                        instrument="piano")  # Use piano forthe Karplus-Strong algorithm

melody_notes = demons_souls_theme_notes()  # Use the new function

melody = synthesize_melody(
    tones, fs, melody_notes,
    base_duration=6,  # Slow and ambient feel
    fade_ratio=0.1  # Smooth note transitions
)

# apply reverb
rir_path = "rirs/RIRS_NOISES/real_rirs_isotropic_noises/air_type1_air_binaural_stairway_1_1_90.wav"
melody = apply_rir(melody, rir_path, fs)

# Save and plot
sf.write("demons_souls_theme_synth_piano.wav", melody, fs)

plt.figure(figsize=(10, 3))
plt.plot(melody[:10000])
plt.title("Demon's Souls Character Creation piano (Karplus-Strong Synth)")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

# Play the sound
import IPython.display as ipd
ipd.Audio("demons_souls_theme_synth_piano.wav")

In [ ]:
# demons souls soul of a mist full song

tones, fs = generate_multi_octave_scale(fs=16000,
                                        duration=1.0,
                                        base_note="C",
                                        base_octave=2,   # Start from C3
                                        octave_range=4,  # Generates notes from C2 to B5
                                        instrument="guitar")  # Use piano forthe Karplus-Strong algorithm

melody_notes = demons_souls_theme_notes()
bass_notes = demons_souls_theme_bass()

melody = synthesize_melody(
    tones, fs, melody_notes,
    base_duration=6,  # Slow and ambient feel
    fade_ratio=0.02
)

bass = synthesize_melody(
    tones, fs, bass_notes,
    base_duration=6,
    fade_ratio=0.02  # Slightly more overlap for ambience
)

# Combine melody and bass
full_theme = melody + bass[:len(melody)]  # Trim in case lengths mismatch slightly

# Apply reverb
rir_path = "rirs/RIRS_NOISES/real_rirs_isotropic_noises/air_type1_air_binaural_stairway_1_1_90.wav"
full_theme = apply_rir(full_theme, rir_path, fs)

# Save and plot
sf.write("demons_souls_theme_full.wav", full_theme, fs)

plt.figure(figsize=(10, 3))
plt.plot(full_theme[:10000])
plt.title("Demon's Souls Character Creation (Full Synth)")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

# Play the sound
ipd.Audio("demons_souls_theme_full.wav")


<p align="center">
  <img src="notes/Majula1.png" width="30%" />
  <img src="notes/Majula2.png" width="30%" />
  <img src="notes/Majula3.png" width="30%" />
</p>


In [ ]:
tones, fs =  generate_multi_octave_scale(fs=16000, duration=1.0, instrument="piano")
melody_notes = majula_theme_melody()

# You can adjust this to control the song speed
melody = synthesize_melody(tones, fs, melody_notes, base_duration=4, fade_ratio=0.2)

# apply reverb
rir_path = "rirs/RIRS_NOISES/real_rirs_isotropic_noises/air_type1_air_binaural_stairway_1_1_90.wav"
melody = apply_rir(melody, rir_path, fs)


# Save to WAV
sf.write("majula_synth.wav", melody, fs)

# Plot waveform
plt.figure(figsize=(10, 3))
plt.plot(melody[:10000])
plt.title("Majula (piano Synth)")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

# Play the sound
import IPython.display as ipd
ipd.Audio("majula_synth.wav")

In [ ]:
# Generate tones (ensure full range for bass)
tones, fs = generate_multi_octave_scale(
    fs=16000,
    duration=1.2,               # slightly longer in case of overlap
    instrument="piano",
    octave_range=5,
    base_note="C",
    base_octave=1              # includes low bass notes
)

# Get all parts
melody_notes = majula_theme_melody()
harmony_notes = majula_theme_harmony()
bass_notes = majula_theme_bass()

# Synthesize all layers
melody = synthesize_melody(tones, fs, melody_notes, base_duration=5, fade_ratio=0.08)
harmony = synthesize_melody(tones, fs, harmony_notes, base_duration=5, fade_ratio=0.09)
bass = synthesize_melody(tones, fs, bass_notes, base_duration=5, fade_ratio=0.1)

# Pad or trim to same length
min_len = min(len(melody), len(harmony), len(bass))
melody = melody[:min_len]
harmony = harmony[:min_len]
bass = bass[:min_len]

# Mix all parts together
combined = melody + harmony + bass

# Apply reverb
rir_path = "rirs/RIRS_NOISES/real_rirs_isotropic_noises/air_type1_air_binaural_stairway_1_1_90.wav"
combined = apply_rir(combined, rir_path, fs)

# Save to file
sf.write("majula_theme_full_synth.wav", combined, fs)

# Plot the waveform
plt.figure(figsize=(10, 3))
plt.plot(combined[:10000])
plt.title("Majula Theme (3-part Piano Synth)")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

# Play the audio
import IPython.display as ipd
ipd.Audio("majula_theme_full_synth.wav")